In [3]:
#giving good output but partially working on images from web

In [5]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

# Define the transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

def load_model(model_path):
    checkpoint = torch.load(model_path)
    con1 = nn.Conv2d(3, 32, 3)
    con2 = nn.Conv2d(32, 64, 3)
    con3 = nn.Conv2d(64, 128, 3)
    fc1 = nn.Linear(26 * 26 * 128, 512)
    fc2 = nn.Linear(512, 256)
    fc3 = nn.Linear(256, 90)

    con1.load_state_dict(checkpoint['con1_state_dict'])
    con2.load_state_dict(checkpoint['con2_state_dict'])
    con3.load_state_dict(checkpoint['con3_state_dict'])
    fc1.load_state_dict(checkpoint['fc1_state_dict'])
    fc2.load_state_dict(checkpoint['fc2_state_dict'])
    fc3.load_state_dict(checkpoint['fc3_state_dict'])

    model = nn.Sequential(
        con1, nn.LeakyReLU(),
        nn.MaxPool2d(2),
        con2, nn.LeakyReLU(),
        nn.MaxPool2d(2),
        con3, nn.LeakyReLU(),
        nn.MaxPool2d(2),
        nn.Flatten(),
        fc1, nn.LeakyReLU(),
        fc2, nn.LeakyReLU(),
        fc3, nn.Sigmoid()
    )

    model.eval()
    return model

def preprocess_image(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    return image

def predict_image(image_path, model):
    image_tensor = preprocess_image(image_path)
    with torch.no_grad():
        output = model(image_tensor)
        _, prediction = torch.max(output, 1)
        return prediction.item()

if __name__ == "__main__":
    model_path = 'run.pth'
    image_path = 'C:\\Users\\srisi\\OneDrive\\Desktop\\sidxt\\hub\\TryCNN\\image.png'
    
    model = load_model(model_path)
    
    result = predict_image(image_path, model)
    print(f"prediction: {result}")


prediction: 2
